In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%load_ext line_profiler

import tensorflow as tf
slim = tf.contrib.slim
tf.logging.set_verbosity(tf.logging.INFO)
sess_config = tf.ConfigProto()
#config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction=0.45

import sys
import os

ADE20K_DATA = '/gpfs01/bethge/data/ADE20K_2016_07_26'
COCO_DATA = '/gpfs01/bethge/share/mscoco/COCO'
PASCAL_VOC_DATA = '/gpfs01/bethge/data/PascalVOC'
MASK_RCNN_MODEL_PATH = 'Mask_RCNN/'
SLIM_MODELS_PATH = '/gpfs01/bethge/home/aecker/lab/projects/tf-models/slim'
TRANSFORMER_MODELS_PATH = '/gpfs01/bethge/home/cmichaelis/tf-models/transformer/'
SIAMESE_MASK_RCNN_PATH = '/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/siamese-mask-rcnn/'

if MASK_RCNN_MODEL_PATH not in sys.path:
    sys.path.append(MASK_RCNN_MODEL_PATH)
if SIAMESE_MASK_RCNN_PATH not in sys.path:
    sys.path.append(SIAMESE_MASK_RCNN_PATH)
    
from samples.coco import coco
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize
    
if SLIM_MODELS_PATH not in sys.path:
    sys.path.append(SLIM_MODELS_PATH)
if TRANSFORMER_MODELS_PATH not in sys.path:
    sys.path.append(TRANSFORMER_MODELS_PATH)
    
import utils as siamese_utils
import model as siamese_model
import siamese_mrcnn_models as model_zoo
    
import time
import random
import numpy as np
import skimage.io
import imgaug
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12.0, 6.0)

from spatial_transformer import transformer

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

In [ ]:
nopascal_classes = [8,10,11,12,13,14,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,60,62,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80]

In [ ]:
pascal_classes = np.array(range(1,81))[np.array([i not in nopascal_classes for i in range(1,81)])]

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1 #6 #12
    LEARNING_RATE = 0.001
    NUM_CLASSES = 80 + 1
    TARGET_MAX_DIM = 96
    TARGET_MIN_DIM = 75
    IMAGE_MIN_DIM = 400
    IMAGE_MAX_DIM = 512
    #IMAGE_RESIZE_MODE = 'none'
    TARGET_SHAPE = np.array([TARGET_MAX_DIM, TARGET_MAX_DIM, 3])
    TARGET_PADDING = True
    MAX_TARGET_INSTANCES = 10
    # Reduce model size for prototyping
    BACKBONE = 'resnet50'
    FPN_FEATUREMAPS = 128
    RPN_ANCHOR_STRIDE = 2
    RPN_TRAIN_ANCHORS_PER_IMAGE = 64
    POST_NMS_ROIS_TRAINING = 250
    POST_NMS_ROIS_INFERENCE = 250
    TRAIN_ROIS_PER_IMAGE = 50
    DETECTION_MAX_INSTANCES = 15
    DETECTION_NMS_THRESHOLD = 0.5
    MAX_GT_INSTANCES = 15
    ACTIVE_CLASSES = np.array(range(1,81))

config = InferenceConfig()
config.display()

In [ ]:
# Load COCO/val dataset
val_dataset = siamese_utils.IndexedCocoDataset()
val_dataset.load_coco(COCO_DATA, "val", year="2017")
val_dataset.prepare()
val_dataset.build_indices()

### Fancy 4

In [ ]:
# One-Shot vs Validation
model_name = ['coco20180529T1829/mask_rcnn_coco_0369.h5']
trainables = ["(res2.*)|(bn2.*)|(res3.*)|(bn3.*)|(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)"]
active_classes = [np.array(range(1,81))]

for i in range(len(models)):
    print("Run {}".format(i+1))
    print("")
    t = time.time()
    print("initializing model...")
    # Create model object in inference mode.
    config.ACTIVE_CLASSES = active_classes[i]
    model = model_zoo.FancySiameseMaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
    model.set_trainable(trainables[i], verbose=0)
    # Load weights
    model.load_weights('/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/logs/' + model_name[i], by_name=True)
    print("evaluating...")
    model.evaluate_dataset(val_dataset, max_images=1000)
    print("")
    print("Fineshed after {:.0f}s".format(time.time()-t))
    print("")
    print("")

### Visualization

In [ ]:
model_name = 'coco20180529T1829/mask_rcnn_coco_0369.h5'
trainables = "(res2.*)|(bn2.*)|(res3.*)|(bn3.*)|(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)"

In [ ]:
model = model_zoo.FancySiameseMaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
model.set_trainable(trainables, verbose=0)
# Load weights
model.load_weights('/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/logs/' + model_name, by_name=True)

In [ ]:
IMAGE_DIR = '/gpfs01/bethge/home/cmichaelis/datasets/COCO/val2017/'

# Load a random image from the images folder
# file_names = next(os.walk(IMAGE_DIR))[2]
# image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
coco_image_ids = [val_dataset.image_info[id]["id"] for id in val_dataset.image_ids]
image_id = random.choice(range(len(coco_image_ids)))
image = val_dataset.load_image(513)#2699)#4673)
print(image_id)

# Load target
category = 1
random_image_id = np.random.choice(val_dataset.category_image_index[category])    
# target_image, _, target_class_ids, target_boxes, _ = \
#     modellib.load_image_gt(val_dataset, config, random_image_id, augmentation=False,
#                   use_mini_mask=config.USE_MINI_MASK)

# box_ind = np.random.choice(np.where(target_class_ids == category)[0])   
# tb = target_boxes[box_ind,:]
# target = target_image[tb[0]:tb[2],tb[1]:tb[3],:]
target = siamese_utils.get_one_target(category, val_dataset, config)


# Run detection
results = model.detect([target], [image], verbose=1)
r = results[0]
siamese_utils.display_siamese_instances(target, image, r['rois'], r['masks'], r['class_ids'], r['scores'])